In [1]:
from models import Darknet
import numpy as np
import torch

In [2]:
# configs
cfg_path80 = 'cfg/yolov3-spp.cfg'
cfg_path3 = 'cfg/yolov3-spp-3cls.cfg'

# models
IMG_SIZE = 416
model80 = Darknet(cfg_path80, IMG_SIZE)
model3 = Darknet(cfg_path3, IMG_SIZE)

# state dict
trained_state_dict = torch.load('weights/yolov3-spp-ultralytics.pt')['model']
model80.load_state_dict(trained_state_dict)

state_dict80 = model80.state_dict()
state_dict3 = model3.state_dict()

Model Summary: 225 layers, 6.29987e+07 parameters, 6.29987e+07 gradients
Model Summary: 225 layers, 6.25841e+07 parameters, 6.25841e+07 gradients


In [3]:
# inference example to make sure that output afterwards is as expected
out = model80(torch.ones((1,3,416,416)))
out[0][0,0,0,0,:]

In [5]:
yolo_layers = [89, 101, 113]  # for spp, found using looking at model80.parameters
class_layers = [str(yolo_layer - 1) for yolo_layer in yolo_layers]  # the convolutions which represent the output including classes
wanted_class_inds = [0, 2, 18]  # person, car, sheep
n_orig_classes = 80
wanted_filters_layer = np.array(list(range(0, 5)) + [ind + 5 for ind in wanted_class_inds]) 
NUM_YOLO_LAYERS = 3

wanted_filters = [j*(n_orig_classes+5)+wanted_filters_layer for j in range(NUM_YOLO_LAYERS)]
wanted_filters = np.concatenate(wanted_filters)

In [6]:
wanted_filters

array([  0,   1,   2,   3,   4,   5,   7,  23,  85,  86,  87,  88,  89,  90,  92, 108, 170, 171, 172, 173, 174, 175, 177, 193])

In [7]:
model80.parameters

<bound method Module.parameters of Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (Conv2d): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.003, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.003, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (Conv2d): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.003, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), strid

In [8]:
wanted_layers = [k for k in state_dict80.keys() if any(layer_name in k for layer_name in class_layers)]

In [9]:
wanted_layers

['module_list.88.Conv2d.weight',
 'module_list.88.Conv2d.bias',
 'module_list.100.Conv2d.weight',
 'module_list.100.Conv2d.bias',
 'module_list.112.Conv2d.weight',
 'module_list.112.Conv2d.bias']

In [10]:
formatted_layers = {layer:state_dict80[layer][wanted_filters] for layer in wanted_layers}

In [11]:
for key in formatted_layers.keys():
    state_dict80[key] = formatted_layers[key]

In [12]:
# assert that we are succesful
model3.load_state_dict(state_dict80)

<All keys matched successfully>

In [13]:
new_out = model3(torch.ones((1,3,416,416)))

In [14]:
new_out[0][0,0,0,0,:]

tensor([ 2.94137,  1.79618, -0.76589, -0.56028, -7.94503, -1.19070, -6.22689, -8.71526], grad_fn=<SliceBackward>)

In [15]:
# save model
torch.save(state_dict80, 'yolo-spp-ultralytics-3class-mars_transfer_model.pt')

In [16]:
# save including other parameters
saved = torch.load('weights/yolov3-spp-ultralytics.pt')
saved['model'] = state_dict80
torch.save(saved, 'weights/yolo-spp-ultralytics-3class-mars_transfer.pt')